# Getting and filtering data for each game for seasons from 2011-2022 (12 seasons)

In [24]:
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoretraditionalv2
import pandas as pd

COUNTER =  1
DIRECTION = "ASC"
LEAGUE = "00"
PLAYER_OR_TEAM = "T"
SEASON_TYPE = "Regular Season"
SORTER = "DATE"

def fetch_season_game_logs(season):
    # Create LeagueGameLog object instance for the season
    gamelog = leaguegamelog.LeagueGameLog(
    COUNTER, DIRECTION, LEAGUE, PLAYER_OR_TEAM, season, SEASON_TYPE, SORTER
)
    
    # Execute request and fetch data
    data = gamelog.get_data_frames()[0] 
    
    # Filter columns and add Home/Away column
    columns_to_keep = ["TEAM_ID","TEAM_NAME","GAME_ID","WL"]
    filtered_data = data[columns_to_keep].copy()
    filtered_data['HOME/AWAY'] = filtered_data['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    
    return filtered_data

def fetch_multiple_seasons(start_year, end_year):
    all_seasons_data = []  # List to store data for all seasons
    
    # Loop through each season from start_year to end_year (inclusive)
    for year in range(start_year, end_year + 1):
        season = f"{year}-{str(year + 1)[-2:]}"  # Format season string (e.g., '2010-11')
        season_data = fetch_season_game_logs(season)
        all_seasons_data.append(season_data)
    
    # Concatenate all seasons' data into a single DataFrame
    combined_data = pd.concat(all_seasons_data, ignore_index=True)
    combined_data = combined_data.sort_values(by='GAME_ID', ascending=True)
    combined_data = combined_data.reset_index(drop=True)

    team_2_id = []
    team_2_name = []

    # Add new columns for team IDs and names from adjacent rows
    for i in range(len(combined_data)):
        if i % 2 == 0:  # Every even row
            team_2_id.append(combined_data.loc[i+1, 'TEAM_ID'])
            team_2_name.append(combined_data.loc[i+1, 'TEAM_NAME'])
        else:  # Every odd row
            team_2_id.append(combined_data.loc[i-1, 'TEAM_ID'])
            team_2_name.append(combined_data.loc[i-1, 'TEAM_NAME'])
    
    # Add new columns to the DataFrame
    combined_data['TEAM_2_ID'] = team_2_id
    combined_data['TEAM_2_NAME'] = team_2_name
    print(team_2_id)
    
    return combined_data

print(fetch_multiple_seasons(2011,2022))

[1610612752, 1610612738, 1610612748, 1610612742, 1610612741, 1610612747, 1610612760, 1610612753, 1610612744, 1610612746, 1610612751, 1610612764, 1610612749, 1610612766, 1610612745, 1610612753, 1610612761, 1610612739, 1610612754, 1610612765, 1610612760, 1610612750, 1610612743, 1610612742, 1610612759, 1610612763, 1610612740, 1610612756, 1610612758, 1610612747, 1610612757, 1610612755, 1610612744, 1610612741, 1610612751, 1610612737, 1610612748, 1610612738, 1610612749, 1610612750, 1610612757, 1610612758, 1610612762, 1610612747, 1610612754, 1610612761, 1610612766, 1610612748, 1610612737, 1610612764, 1610612765, 1610612739, 1610612760, 1610612763, 1610612740, 1610612738, 1610612759, 1610612746, 1610612762, 1610612743, 1610612756, 1610612755, 1610612752, 1610612744, 1610612751, 1610612753, 1610612759, 1610612745, 1610612760, 1610612742, 1610612741, 1610612758, 1610612743, 1610612757, 1610612747, 1610612752, 1610612753, 1610612766, 1610612739, 1610612754, 1610612738, 1610612765, 1610612737, 161

# Combining Win/Loss data with overall Season Stats 

In [10]:
from api_helpers.team_stats_helpers import load_dataframe

nba_dataframe = load_dataframe(["PTS","FGM","FGA","FG_PCT","FG3M","FG3A","FG3_PCT","FTM","FTA","FT_PCT","OREB","DREB","AST"])
nba_dataframe = nba_dataframe.drop(nba_dataframe[nba_dataframe["FGM"] == 0].index)
nba_dataframe.sort_values(by='YEAR')

nba_dataframe['YEAR'] = nba_dataframe['YEAR'].str.split('-').str[0]
nba_dataframe['YEAR'] = pd.to_numeric(nba_dataframe['YEAR'])
nba_dataframe.sort_values(by="YEAR",inplace=True)

nba_dataframe["NBA_FINALS_APPEARANCE"].fillna(0.0, inplace=True)
nba_dataframe["NBA_FINALS_APPEARANCE"].replace("FINALS APPEARANCE", 0.0, inplace=True)
nba_dataframe["NBA_FINALS_APPEARANCE"].replace("LEAGUE CHAMPION", 2, inplace=True)

nba_dataframe = nba_dataframe[nba_dataframe['YEAR'] >= 2011]
nba_dataframe = nba_dataframe.reset_index(drop=True)

print(nba_dataframe)


        TEAM_ID  YEAR  NBA_FINALS_APPEARANCE   FGM   FGA  FG_PCT  FG3M  FG3A  \
0    1610612759  2011                    0.0  2611  5463   0.478   552  1405   
1    1610612747  2011                    0.0  2429  5319   0.457   362  1112   
2    1610612746  2011                    0.0  2439  5366   0.455   514  1441   
3    1610612748  2011                    2.0  2446  5212   0.469   370  1030   
4    1610612739  2011                    0.0  2261  5360   0.422   440  1273   
..          ...   ...                    ...   ...   ...     ...   ...   ...   
385  1610612765  2023                    0.0  3353  7236   0.463   906  2602   
386  1610612756  2023                    0.0  3482  7063   0.493  1020  2671   
387  1610612740  2023                    0.0  3484  7165   0.486  1023  2673   
388  1610612764  2023                    0.0  3523  7493   0.470  1015  2915   
389  1610612766  2023                    0.0  3281  7133   0.460   989  2788   

     FG3_PCT   FTM   FTA  FT_PCT  OREB 